In [1]:
#importing required libraries
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
pd.set_option('display.max_columns', None)
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

link to dataset summary-
https://www.sciencedirect.com/science/article/pii/S2352340918315191

In [2]:
#loading dataset
hotels_df= pd.read_csv("/kaggle/input/hotel-booking/hotel_booking.csv")
hotels_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [3]:
#overview
hotels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 36 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

# Cleaning the dataset

In [4]:
#checking the number of null values in the dataset
hotels_df.isna().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [5]:
#dropping unnecessary columns(email, phone-number, credit_card)
hotels_df.drop(["email", "phone-number", "credit_card"], axis= 1, inplace= True)

In [6]:
#filling all null values in children column with the mean value
hotels_df["children"].fillna(round(hotels_df["children"].mean(), 0), inplace= True)

In [7]:
#replacing null values in agent column with 0, representing the client himself/herself made the booking
hotels_df["agent"].fillna(0, inplace= True)

In [8]:
#replacing null values in company column with 0, representing the client is not affiliated to any particular company, but rather visiting on personal capacity
hotels_df["company"].fillna(0, inplace= True)

In [9]:
#replacing null values in country column with the mode value, as country is categorical variable
hotels_df["country"].fillna(hotels_df["country"].mode()[0], inplace= True)

In [10]:
#converting children, agent, company to appropriate number format(int64)
hotels_df.loc[:, ["children", "agent", "company"]]= hotels_df.loc[:, ["children", "agent", "company"]].astype("int64")

In [11]:
#converting categorical variables from object type to category type
categorical_variables= ["is_canceled", "meal", "country", "market_segment", "distribution_channel", "is_repeated_guest", "reserved_room_type", "assigned_room_type", "deposit_type", "agent", "company", "customer_type", "reservation_status"]

hotels_df.loc[:, categorical_variables]= hotels_df.loc[:, categorical_variables].astype("category")

In [12]:
hotels_df.arrival_date_month.unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [13]:
cat_type = CategoricalDtype(categories=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"], ordered=True)

hotels_df.loc[:, "arrival_date_month"]= hotels_df.loc[:, "arrival_date_month"].astype(cat_type)

In [14]:
def date(row):
    '''
    Input- row
    Output- datetime object
    '''
    
    date_str= f'{row["arrival_date_year"]}/{row["arrival_date_month"]}/{row["arrival_date_day_of_month"]}'
    return datetime.strptime(date_str, "%Y/%B/%d")

In [15]:
#adding date column to df
hotels_df["arrival_date"]= hotels_df.apply(lambda row: date(row), axis= 1)

In [16]:
def days_spent(row):
    '''
    Input- row
    Output- int object specifying the total number of days the guest stayed at the hotel
    '''
    return sum((row["stays_in_weekend_nights"], row["stays_in_week_nights"]))

In [17]:
hotels_df.loc[5]

hotel                                    Resort Hotel
is_canceled                                         0
lead_time                                          14
arrival_date_year                                2015
arrival_date_month                               July
arrival_date_week_number                           27
arrival_date_day_of_month                           1
stays_in_weekend_nights                             0
stays_in_week_nights                                2
adults                                              2
children                                            0
babies                                              0
meal                                               BB
country                                           GBR
market_segment                              Online TA
distribution_channel                            TA/TO
is_repeated_guest                                   0
previous_cancellations                              0
previous_bookings_not_cancel

In [18]:
days_spent(hotels_df.loc[5])

2

In [19]:
def total_spending(row):
    '''
    Input- row
    Output- float object representing the total spending the guest made
    '''
    return row["adr"]* days_spent(row)

In [20]:
#adding days_spent column to df
hotels_df["days_spent"]= hotels_df.apply(lambda row: days_spent(row), axis= 1)

In [21]:
#adding total spending column to df
hotels_df["total_spending"]= hotels_df.apply(lambda row: total_spending(row), axis= 1)

In [22]:
hotels_df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'name', 'arrival_date',
       'days_spent', 'total_spending'],
      dtype='object')

In [23]:
def total_guests(row):
    '''
    Input- row
    Output- int object specifying the number of guests staying in the room
    '''
    return sum((row["adults"], row["children"], row["babies"]))

In [24]:
hotels_df["total_guests"]= hotels_df.apply(lambda row: total_guests(row), axis= 1)

In [25]:
hotels_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,arrival_date,days_spent,total_spending,total_guests
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,2015-07-01,0,0.0,2
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,2015-07-01,0,0.0,2
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,0,0,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,2015-07-01,1,75.0,1
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,0,0,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,2015-07-01,1,75.0,1
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,2015-07-01,2,196.0,2


# Descriptive Statistics

In [26]:
#numerical variables
hotels_df.describe()

,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,days_spent,total_spending,total_guests
count,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000,119390.000000
mean,104.011416,2016.156554,27.165173,15.798241,0.927599,2.500302,1.856403,0.103886,0.007949,0.087118,0.137097,0.221124,2.321149,101.831122,0.062518,0.571363,3.427900,357.848208,1.968239
std,106.863097,0.707476,13.605138,8.780829,0.998613,1.908286,0.579261,0.398555,0.097436,0.844336,1.497437,0.652306,17.594721,50.535790,0.245291,0.792798,2.557439,335.910109,0.722394
min,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.380000,0.000000,0.000000,0.000000,-63.800000,0.000000
25%,18.000000,2016.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,69.290000,0.000000,0.000000,2.000000,146.000000,2.000000
50%,69.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,94.575000,0.000000,0.000000,3.000000,267.000000,2.000000
75%,160.000000,2017.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,126.000000,0.000000,1.000000,4.000000,446.250000,2.000000
max,737.000000,2017.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,26.000000,72.000000,21.000000,391.000000,5400.000000,8.000000,5.000000,69.000000,7590.000000,55.000000


In [27]:
#categorical variables
hotels_df.describe(include= "category")

,is_canceled,arrival_date_month,meal,country,market_segment,distribution_channel,is_repeated_guest,reserved_room_type,assigned_room_type,deposit_type,agent,company,customer_type,reservation_status
count,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390,119390
unique,2,12,5,177,8,5,2,10,12,3,334,353,4,3
top,0,August,BB,PRT,Online TA,TA/TO,0,A,A,No Deposit,9,0,Transient,Check-Out
freq,75166,13877,92310,49078,56477,97870,115580,85994,74053,104641,31961,112593,89613,75166


In [28]:
#creating to seprate datasets for the two different hotels
#resort hotel
resort_df= hotels_df.loc[hotels_df["hotel"]== "Resort Hotel"].reset_index(drop= True)
resort_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,arrival_date,days_spent,total_spending,total_guests
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,2015-07-01,0,0.0,2
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,2015-07-01,0,0.0,2
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,0,0,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,2015-07-01,1,75.0,1
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,0,0,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,2015-07-01,1,75.0,1
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,2015-07-01,2,196.0,2


In [29]:
#city hotel
city_df= hotels_df.loc[hotels_df["hotel"]== "City Hotel"].reset_index(drop= True)
city_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,arrival_date,days_spent,total_spending,total_guests
0,City Hotel,0,6,2015,July,27,1,0,2,1,0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,0,0,Transient,0.0,0,0,Check-Out,2015-07-03,Elizabeth Ross,2015-07-01,2,0.0,1
1,City Hotel,1,88,2015,July,27,1,0,4,2,0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,76.5,0,1,Canceled,2015-07-01,Adam Aguilar,2015-07-01,4,306.0,2
2,City Hotel,1,65,2015,July,27,1,0,4,1,0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,68.0,0,1,Canceled,2015-04-30,Mark Keller,2015-07-01,4,272.0,1
3,City Hotel,1,92,2015,July,27,1,2,4,2,0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,76.5,0,2,Canceled,2015-06-23,Sarah Scott,2015-07-01,6,459.0,2
4,City Hotel,1,100,2015,July,27,2,0,2,2,0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,0,0,Transient,76.5,0,1,Canceled,2015-04-02,Misty Perry,2015-07-02,2,153.0,2


Assumptions-

1.80% of the business came from 20% of agents/companies
    
    1.1. find the top 10 agents
    1.2. find the top 10 companies (based on total spending by the guests)

2.Bookings are more on weekends compared to weekdays

3.City hotels are more prone to canclellations compared to resort hotels

4.Corporate clients are more prone to cancel their stays

5.Resort Hotels see higher business during holiday seasons, whereas for City Hotels, the clientele is pretty much same thru out the year

6.Most Profiatble Guests(as per ADR * total no.of nights spent)

7.Most preferred meal plan

8.Relation b/w meal plan and ADR

9.In which month do both of the hotels have the least/highest occupancy

10.Count of guests based on countries(Map Visualization)



Visualizations-

1.Calendar based Heat Map

2.Calendar based Heat Map to check when their are least transient/transient-party guests
Incentivize contract bookings at this time of the year

3.

In [30]:
hotels_df["customer_type"].unique()

['Transient', 'Contract', 'Transient-Party', 'Group']
Categories (4, object): ['Contract', 'Group', 'Transient', 'Transient-Party']

https://www.xotels.com/en/glossary/displacement-analysis

use displacement analysis to check which is more profitable for the two groups of hotels(resort/city). take into account the month/season.

In [31]:
for hotel_type in hotels_df["hotel"].unique():
    print(hotel_type)
    print("weekend")
    print(hotels_df[hotels_df["hotel"]== hotel_type]["stays_in_weekend_nights"].mean())
    print("weekday")
    print(hotels_df[hotels_df["hotel"]== hotel_type]["stays_in_week_nights"].mean())

Resort Hotel
weekend
1.1898152770843735
weekday
3.1287319021467797
City Hotel
weekend
0.7951846716248582
weekday
2.1829572671120636


In [32]:
len(city_df[city_df["customer_type"]== "Contract"])

2300

In [33]:
len(resort_df[resort_df["customer_type"]== "Contract"])

1776

In [34]:
hotels_df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'name', 'arrival_date',
       'days_spent', 'total_spending', 'total_guests'],
      dtype='object')

In [35]:
hotels_df.loc[hotels_df["total_spending"]<0]

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,arrival_date,days_spent,total_spending,total_guests
14969,Resort Hotel,0,195,2017,March,10,5,4,6,2,0,0,BB,GBR,Groups,Direct,1,0,2,A,H,2,No Deposit,273,0,0,Transient-Party,-6.38,0,0,Check-Out,2017-03-15,Chase Santos,2017-03-05,10,-63.8,2


In [36]:
xyz= hotels_df.loc[hotels_df["total_guests"]==0]
xyz.loc[xyz["total_spending"]!=0]["days_spent"]

31765     10
47444      6
50701      6
50702      6
55809      8
78586      3
85931      8
88439      6
92814      7
92962      2
94769      4
95544      4
96648      9
98492     13
101399     2
101401     4
101794    57
101957     4
102008     3
102185    11
103714     3
104145     7
104516    14
106562    15
106833     3
107594    11
111406     1
114583     5
115029     3
116251     2
116534     7
Name: days_spent, dtype: int64

In [37]:
hotels_df["agent"]!= 0

0         False
1         False
2         False
3          True
4          True
          ...  
119385     True
119386     True
119387     True
119388     True
119389     True
Name: agent, Length: 119390, dtype: bool

In [38]:
#find the top 10 agents

agents_df= hotels_df.loc[hotels_df["agent"]!= 0]

agents_df.shape

(103050, 37)

In [39]:
agents_df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,arrival_date,days_spent,total_spending,total_guests
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,0,0,Transient,75.00,0,0,Check-Out,2015-07-02,Laura Murray,2015-07-01,1,75.00,1
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.00,0,1,Check-Out,2015-07-03,Linda Hines,2015-07-01,2,196.00,2
5,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.00,0,1,Check-Out,2015-07-03,Jasmine Fletcher,2015-07-01,2,196.00,2
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,0,0,FB,PRT,Direct,Direct,0,0,0,C,C,0,No Deposit,303,0,0,Transient,103.00,0,1,Check-Out,2015-07-03,William Velez,2015-07-01,2,206.00,2
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,82.00,0,1,Canceled,2015-05-06,Steven Murphy,2015-07-01,3,246.00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394,0,0,Transient,96.14,0,0,Check-Out,2017-09-06,Claudia Johnson,2017-08-30,7,672.98,2
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9,0,0,Transient,225.43,0,2,Check-Out,2017-09-07,Wesley Aguilar,2017-08-31,7,1578.01,3
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9,0,0,Transient,157.71,0,4,Check-Out,2017-09-07,Mary Morales,2017-08-31,7,1103.97,2
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89,0,0,Transient,104.40,0,0,Check-Out,2017-09-07,Caroline Conley MD,2017-08-31,7,730.80,2
